<a href="https://colab.research.google.com/github/RMoulla/TP3/blob/main/LSTM_Classification_Correction_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entraînement d'un modèle LSTM pour la Classification de texte

Dans ce tutoriel, nous allons aborder les étapes essentielles pour construire et entraîner un modèle de réseau de neurones récurrents, spécifiquement une architecture LSTM (Long Short-Term Memory), pour la classification de texte en utilisant le jeu de données AG_NEWS.

### Dataset

Le jeu de données AG_NEWS est un ensemble de données de classification des nouvelles qui comprend des titres d'articles de nouvelles et des descriptions provenant de plus de 2 000 sources d'informations. Il est organisé en quatre catégories principales : Monde, Sports, Business et Science/Technologie, ce qui en fait un choix excellent pour les tâches de classification de texte dans le domaine de la reconnaissance automatique des sujets de nouvelles. Chaque enregistrement est étiqueté avec son appartenance à l'une de ces catégories, offrant ainsi un cadre clair pour les modèles supervisés d'apprentissage automatique.



### Préparation de l'environnement  
Nous commencerons par configurer notre environnement de développement en important les bibliothèques nécessaires fournies par PyTorch, y compris les modules pour les jeux de données, la modélisation, et l'optimisation.

### Chargement des données
Nous chargerons le jeu de données AG_NEWS en utilisant les utilitaires de torchtext, qui simplifient le processus de téléchargement et de prétraitement des données textuelles.

### Prétraitement du texte
Nous aborderons ensuite les étapes de tokenisation et de construction de vocabulaire, deux étapes préliminaires cruciales pour transformer le texte brut en une forme numérique que notre modèle pourra interpréter.

### Construction du modèle LSTM
Nous définirons la classe LSTMClassifier, qui établira l'architecture de notre modèle LSTM en utilisant les couches et fonctions fournies par PyTorch.

### Entraînement du modèle  
Nous procéderons à l'entraînement du modèle en utilisant la rétropropagation à travers le temps (Backpropagation Through Time, BPTT), tout en gérant les aspects tels que la fonction de perte et l'optimisation.

### Évaluation du modèle  
Nous évaluerons les performances de notre modèle en calculant la précision sur un ensemble de test, ce qui nous donnera une indication de la façon dont notre modèle est susceptible de se comporter en production.


In [ ]:
!pip install portalocker>=2.0.0

In [ ]:
from torchtext.datasets import AG_NEWS

# Crée un itérateur
train_iter = iter(AG_NEWS(split='train'))

# Récupère le premier exemple
first_example = next(train_iter)

print(first_example)

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


In [ ]:

tokenizer = get_tokenizer('basic_english')
tokenizer(first_example[1])

['wall',
 'st',
 '.',
 'bears',
 'claw',
 'back',
 'into',
 'the',
 'black',
 '(',
 'reuters',
 ')',
 'reuters',
 '-',
 'short-sellers',
 ',',
 'wall',
 'street',
 "'",
 's',
 'dwindling\\band',
 'of',
 'ultra-cynics',
 ',',
 'are',
 'seeing',
 'green',
 'again',
 '.']

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch import nn
import torch.optim as optim

# Détecte si un GPU est disponible et le sélectionne pour les calculs, sinon utilise le CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Charge les ensembles de données d'entraînement et de test AG News.
train_iter, test_iter = AG_NEWS()

# Initialise le tokenizer pour la tokenisation en anglais basique.
tokenizer = get_tokenizer('basic_english')

# Définit une fonction génératrice qui itère sur l'ensemble de données et tokenize le texte.
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

# Construit un vocabulaire à partir de l'itérateur de tokens avec un token inconnu spécial "<unk>".
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Définit une classe de modèle pour le classificateur LSTM en utilisant le module nn de PyTorch.
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Couche d'embedding
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)  # Couche LSTM
        self.fc = nn.Linear(hidden_dim, num_classes)  # Couche de sortie

    def forward(self, text):
        # Définit le flux de données à travers le réseau.
        embedded = self.embedding(text)  # Passe le texte tokenisé à travers la couche d'embedding
        lstm_out, _ = self.lstm(embedded)  # Passe les embeddings à travers la couche LSTM
        final_out = lstm_out[:, -1, :]  # Utilise la dernière sortie cachée de la séquence
        return self.fc(final_out)  # Passe la dernière sortie à travers la couche fully-connected

# Initialise les paramètres du modèle.
vocab_size = len(vocab)
embed_dim = 64
hidden_dim = 128
num_classes = 4  # Les catégories sont Monde, Sports, Business, Science/Technologie

# Crée une instance du modèle LSTMClassifier et la transfère sur le périphérique sélectionné (GPU ou CPU).
model = LSTMClassifier(vocab_size, embed_dim, hidden_dim, num_classes).to(device)

# Définit la fonction de perte CrossEntropy pour la classification et l'optimiseur Adam.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prépare les données pour l'entraînement avec une fonction de collation personnalisée.
def collate_batch(batch):
    """
    Prépare un lot pour l'entraînement ou l'évaluation.

    Cette fonction est utilisée comme fonction de collation pour le DataLoader. Elle traite chaque lot
    en séparant les étiquettes des textes, tokenisant les textes, convertissant les tokens en indices
    via le vocabulaire, puis padde les séquences pour qu'elles aient toutes la même longueur.

    Args:
    batch (list): Un lot d'échantillons provenant du DataLoader, où chaque échantillon est un tuple
                  contenant l'étiquette et le texte brut.

    Returns:
    Tuple[Tensor, Tensor]: Le premier tensor contient les étiquettes du lot, et le second tensor
                           contient les textes tokenisés et paddés du lot, tous deux prêts à être
                           passés à travers le modèle.
    """
    label_list, text_list = [], []  # Initialisation des listes pour stocker séparément les étiquettes et les textes traités.

    for (_label, _text) in batch:
        label_list.append(_label - 1)  # Ajuste les étiquettes pour la perte de CrossEntropy et les ajoute à la liste des étiquettes.
        processed_text = torch.tensor(vocab(tokenizer(_text)), dtype=torch.int64)  # Tokenise le texte, convertit les tokens en indices, et crée un tensor.
        text_list.append(processed_text)  # Ajoute le texte traité à la liste des textes.

    # Convertit les listes d'étiquettes et de textes en tensors, puis les transfère sur le dispositif approprié (GPU ou CPU).
    # Padded les séquences de textes pour qu'elles aient toutes la même longueur.
    return torch.tensor(label_list, dtype=torch.int64).to(device), nn.utils.rnn.pad_sequence(text_list, batch_first=True).to(device)


# Crée des chargeurs de données pour l'entraînement et les tests.
train_loader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

# Définit une fonction pour entraîner le modèle.
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, device):
    """
    Entraîner le modèle LSTM avec les données fournies.

    Args:
    model (torch.nn.Module): Le modèle LSTM à entraîner.
    train_loader (DataLoader): Chargeur de données pour l'ensemble d'entraînement.
    test_loader (DataLoader): Chargeur de données pour l'ensemble de test.
    criterion (loss_function): Fonction de perte à utiliser pour l'entraînement.
    optimizer (torch.optim): Optimiseur pour mettre à jour les poids du modèle.
    num_epochs (int): Nombre d'époques d'entraînement.
    device (torch.device): Dispositif sur lequel le modèle est exécuté (CPU ou GPU).

    Returns:
    None
    """
    for epoch in range(num_epochs):  # Boucle sur le nombre d'époques spécifié.
        model.train()  # Met le modèle en mode entraînement.
        total_loss = 0  # Accumulateur pour la perte totale sur l'ensemble d'entraînement.
        num_batches = 0  # Compteur pour le nombre de lots traités.

        for labels, text in train_loader:
            num_batches += 1  # Incrémente le compteur de lots.
            labels, text = labels.to(device), text.to(device)  # Transfère les données sur le dispositif approprié.
            optimizer.zero_grad()  # Réinitialise les gradients de l'optimiseur pour le nouveau lot.
            output = model(text)  # Calcule la sortie du modèle pour le lot actuel.
            loss = criterion(output, labels)  # Calcule la perte entre les prédictions et les étiquettes réelles.
            loss.backward()  # Calcule les gradients de la perte par rapport aux poids du modèle.
            optimizer.step()  # Met à jour les poids du modèle en fonction des gradients calculés.
            total_loss += loss.item()  # Ajoute la perte du lot à la perte totale pour cette époque.

        # Calcule la perte moyenne d'entraînement sur tous les lots pour l'époque.
        train_loss = total_loss / num_batches
        # Évalue le modèle sur l'ensemble de test pour obtenir la perte et la précision du test.
        test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, device)
        # Affiche les statistiques de performance après chaque époque.
        print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


def evaluate_model(model, test_loader, criterion, device):
    """
    Évaluer le modèle sur l'ensemble de test.

    Args:
    model (torch.nn.Module): Le modèle LSTM à évaluer.
    test_loader (DataLoader): Chargeur de données pour l'ensemble de test.
    criterion (loss_function): Fonction de perte utilisée pour l'évaluation.
    device (torch.device): Dispositif sur lequel le modèle est exécuté (CPU ou GPU).

    Returns:
    float: La perte moyenne sur l'ensemble de test.
    float: La précision du modèle sur l'ensemble de test.
    """
    model.eval()  # Met le modèle en mode évaluation pour désactiver le dropout ou la normalisation par lots, si présent.
    total_loss = 0  # Accumulateur pour la perte totale sur l'ensemble de test.
    correct_preds = 0  # Compteur pour le nombre total de prédictions correctes.
    total_samples = 0  # Compteur pour le nombre total d'échantillons dans l'ensemble de test.

    with torch.no_grad():  # Désactive la génération de gradients pour les opérations suivantes, ce qui économise de la mémoire et des calculs.
        for labels, text in test_loader:
            labels, text = labels.to(device), text.to(device)  # Transfère les données sur le dispositif approprié.
            output = model(text)  # Obtient les prédictions du modèle pour le lot actuel.
            loss = criterion(output, labels)  # Calcule la perte entre les prédictions et les étiquettes réelles.
            total_loss += loss.item()  # Ajoute la perte du lot à la perte totale.
            predictions = output.argmax(1)  # Obtient les indices des prédictions maximales (classes prédites).
            correct_preds += (predictions == labels).sum().item()  # Incrémente le nombre de prédictions correctes.
            total_samples += labels.size(0)  # Ajoute le nombre d'échantillons dans le lot au nombre total d'échantillons.

    # Calcule la perte moyenne et la précision sur l'ensemble de test.
    return total_loss / total_samples, correct_preds / total_samples


# Entraîne et évalue le modèle
num_epochs = 5  # Définit le nombre d'itérations pour l'entraînement.
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, device)

Epoch 1, Train Loss: 0.657854645920669, Test Loss: 0.04242232294863482, Test Accuracy: 0.8890789473684211
Epoch 2, Train Loss: 0.23383610082389786, Test Loss: 0.038050668389524187, Test Accuracy: 0.9048684210526315
Epoch 3, Train Loss: 0.16286698982386152, Test Loss: 0.04099694360566852, Test Accuracy: 0.905921052631579
Epoch 4, Train Loss: 0.1146316030843164, Test Loss: 0.043195627161254184, Test Accuracy: 0.9057894736842105
Epoch 5, Train Loss: 0.07919016900598216, Test Loss: 0.04685469420233044, Test Accuracy: 0.9069736842105263
